In [1]:
import tweepy as tw
import os
import json
import random

In [3]:
#Twitter API credentials)
#/////--------------------DA CAMBIARE-----------------------/////
api_key = "FzUscQQNsdWhkxebVGFZyuHsh"
api_secret = "Tr1OqulWQIYYI6aEpWVOb7U5NL9DoRtCAviXh4tYg4LXg9sBZe"
access_token = "1040603971637923845-9uZqYh42gsCmwoDCB9nTcoSHrHwiV8"
access_secret = "OsZL0PZC9WDw6Xwcl2ASpIisyfkKW3Bwu4BomNgWxAcj0"
bearer_token = "AAAAAAAAAAAAAAAAAAAAAJQUJQEAAAAA%2B5CTzK8KotMS%2FuB61Dlul7ijBE4%3D5uWZb7w1FPD6nb4SpvmnZ4hxGjqGwzfMquWPHQQooS3jkgNg67"

auth = tw.OAuthHandler(api_key, api_secret)
auth.set_access_token(access_token, access_secret)
api = tw.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

if api.verify_credentials:
    print ('Authentication completed successfully!')

Authentication completed successfully!


In [4]:
data_folder = "data"

def serialize_json(folder, filename, data):
    if not os.path.exists(folder):
        os.makedirs(folder, exist_ok=True)
    with open(f"{folder}/{filename}", 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
        f.close()
    print(f"Data serialized to path: {folder}/{filename}")

def read_json(path):
    if os.path.exists(path):
        with open(path, "r", encoding="utf8") as file:
            data = json.load(file)
        print(f"Data read from path: {path}")
        return data
    else:
        print(f"No data found at path: {path}")
        return {}

1. Scaricamento utenti followers ( api.followers ) e utenti following (api.friends ) dei seguenti
    cinque account (i dati numerici potrebbero subire minime variazioni):

    ○ @mizzaro <br>
    ○ @damiano10 <br>
    ○ @Miccighel_ <br>
    ○ @eglu81 <br>
    ○ @KevinRoitero


In [5]:
accounts = ["mizzaro" , "damiano10", "Miccighel_", "elgu81", "KevinRoitero"]
followers = [157, 786, 332, 539, 103]
followings = [331, 837, 212, 621, 256]

accounts --> vettore contenenti gli screen_name dei quali si vogliono scaricare i dati (vettore)<br>
what --> specifica il cosa voglio scaricare (stringa) <br>
quantity --> quantità di followers/followings si vogliono scaricare (intero) <br>
operation --> funzione dell'API da chiamere (funzione)

In [27]:
def download(accounts, what, quantity, operation):
    i = 0
    for account in accounts:
        data= []
        print(f"Downloading {what} di {account}")

        for item in tw.Cursor(
                operation,
                screen_name=account,
                skip_status=True,
                include_user_entities=False
            ).items(quantity[i]):

            retriveData(item, data)

        serialize_json(data_folder, f"{account}_{what}s.json", data)
        i += 1

def retriveData(item, data):
    json_data = item._json
    found_follower = {}

    found_follower["id"] = json_data["id"]
    found_follower["name"] = json_data["name"]
    found_follower["screen_name"] = json_data["screen_name"]
    found_follower["location"] = json_data["location"]

    data.append(found_follower)

## Download follower

In [23]:
download(accounts, "follower", followers, api.followers)

Data serialized to path: data/francemattee follower.json
Data serialized to path: data/Martiviola follower.json


## Download following

In [26]:
download(accounts, "following", followings, api.friends)

Data serialized to path: data/francemattee following.json
Data serialized to path: data/Martiviola following.json


2. Scegliete 5 utenti followers a caso tra quelli di ciascuno dei cinque account e
    scaricate ulteriori 10 utenti followers ( followers dei followers )


In [20]:
#for account in accounts:
#followersOfAccount = read_json(f"{data_folder}/{account}_followers")
# for i in range(quantity):
def rndIndex(quantity, max):
    rndNum = []

    while len(rndNum) < quantity:
        pos = random.randint(0, max)

        if pos not in rndNum:
            rndNum.append(pos)
    return rndNum

#DA VERIFICARE FUNZIONAMENTO
def retriveAccounts(rnd):
    data = read_json(f"{data_folder}/{account}_followers")
    newAccounts = []

    for index in rnd:
        #controllo che aaccount selezionato abbia più di 10 followers
        item = data[index]

        if len(api.followers_ids(item["screen_name"]).Cursor(10)) >= 10 : # non sono sicuro che il controllo funzioni
            newAccounts.append(item["screen_name"])# DA CONTROLLARE IL COMPO DA PRENDERE IN CONSIDERAZIONE

    return newAccounts

SyntaxError: invalid syntax (<ipython-input-20-5b04b3169b93>, line 22)

## Download followers of followers

In [ ]:
i = 0
for account in accounts:
    max = followers[i]

    rndNum = rndIndex(5, max)

    download(retriveAccounts(rndNum),
             f"{account}_followersOfFollower", 10, api.followers)
    i += 1

3. Scegliete 5 utenti following a caso tra quelli di ciascuno dei cinque account e
    scaricate ulteriori 10 utenti following ( following dei following )

## Download followings of followings

In [ ]:
i = 0
for account in accounts:
    rndNum = rndIndex(5, i)

    download(retriveAccounts(rndNum),
             f"{account}_followingsOfFollowing", 10, api.friends)
    i += 1

4. Scaricare i dettagli del profilo di tutti gli utenti recuperati


5. Costruite la rete sociale (grafo):

    ○ Inserite l’ id di ciascun utente come identificatore del nodo <br>
    ○ Ogni arco rappresenta una relazione follows tra due utenti <br>
    ○ Inserite i dettagli del profilo di ciascun utente come attributi del nodo <br>
    ○ Inserite i membri del vostro gruppo come attributi del grafo <br>
    ○ Per ogni nodo, aggiungete un attributo con il numero di follower individuati


6. Producete una visualizzazione interattiva del grafo usando pyvis


7. Verificate se il grafo:

    ○ è connesso ( is_connected ) <br>
    ○ è bipartito ( is_bipartite )


8. Misurate le seguenti distanze sul grafo:

    ○ Centro ( center ) <br>
    ○ Diametro ( diameter ) <br>
    ○ Raggio ( radius )


9. Calcolate le seguenti misure di centralità sul grafo:

    ○ Betweenness centrality ( betweenness_centrality ) <br>
    ○ Closeness centrality ( closeness_centrality ) <br>
    ○ Degree centrality ( degree_centrality ) <br>
    ○ In-degree centrality ( in_degree_centrality ) <br>
    ○ Out-degree centrality ( out_degree_centrality ) <br>
    ○ Page Rank ( pagerank ) <br>
    ○ HITS ( hits )


10. Generate il sottografo indotto dal nodo damiano10 ( ego_graph ) e calcolate:

    ○ Cricca massima ( max_clique ) <br>
    ○ Dimensione della cricca massima ( large_clique_size )


11. Calcolate la copertura minima degli archi ( min_edge_cover ) del grafo


12. Calcolate i seguenti coefficienti per stimare la “small-world-ness” del grafo:

    ○ Coefficiente omega ( omega ) <br>
    ○ Coefficiente sigma ( sigma )

13. Calcolare la correlazione di Pearson Rho e di Kendall Tau fra le misure di centralità;
    riportare il risultato in due tabelle